In [2]:
# import libraries (including our short d3_lib script)
from IPython.core.display import HTML
import d3_lib, random
import re
import json

### TO hierarchy

In [2]:
%%script bash
arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  

#CONSTRUCT 
#{
#    ?x rdfs:subClassOf ?o .
#}
SELECT ?x ?xlabel ?o ?olabel
WHERE
{
    ?x rdfs:subClassOf ?o .
    filter(!isblank(?o)) 
    filter(!regex(str(?o), "PATO"))
    filter(regex(str(?o), "TO"))
    ?x rdfs:label ?xlabel . 
    ?o rdfs:label ?olabel .
}' > to.json

### create the JSON file from TO

In [61]:
with open('to.json') as data_file:    
    data = json.load(data_file)
    
    list = data["results"]["bindings"]
    
    graph = {"nodes": [], "links": []}
    #force plant trait to be he first node
    graph["nodes"].append( {"name": "plant trait", "value": 0.5, "group": 9, "id": "http://purl.obolibrary.org/obo/TO_0000387" } )
    for entry in list:
        x = 1
        o = 1
        #check if the nodes already exist in the graph
        for e in graph["nodes"]:
            if e["name"] == entry["xlabel"]["value"]:
                x=graph["nodes"].index(e)
            if e["name"] == entry["olabel"]["value"]:
                o=graph["nodes"].index(e)
        # if the term does not exist, create it        
        if x==1:
            graph["nodes"].append( {"name": entry["xlabel"]["value"], "value": 0.5, "group": 9, "id":entry["x"]["value"] } )
            x=len(graph["nodes"])-1
        if o==1:
            graph["nodes"].append( {"name": entry["olabel"]["value"], "value": 0.5, "group": 9, "id":entry["o"]["value"]} )
            o=len(graph["nodes"])-1
            
        # create the link
        if  entry["x"]["value"] != entry["o"]["value"]:
            graph["links"].append( {"source": x, "target": o, "value": 5} )



### add CO mappings

In [62]:
with open('co.json') as data_file:    
    co = json.load(data_file)
    
    #create the nodes
    for entry in co:
        if "CO_341" in entry["id"]:
            entry["group"] = 1
        if "CO_336" in entry["id"]:
            entry["group"] = 2
        if "CO_322" in entry["id"]:
            entry["group"] = 3
        if "CO_321" in entry["id"]:
            entry["group"] = 4
        if "CO_331" in entry["id"]:
            entry["group"] = 5
        if "CO_339" in entry["id"]:
            entry["group"] = 6
        if "CO_334" in entry["id"]:
            entry["group"] = 7
        if "CO_320" in entry["id"]:
            entry["group"] = 8
            
        entry["value"] = 1
        graph["nodes"].append(entry)

In [63]:
with open('COTOMappings.json') as data_file:    
    coto = json.load(data_file)
    
    #create the links
    for entry in coto:
        #get the indexes of existing nodes
        for e in graph["nodes"]:
            if e["id"] == entry["x"]:
                x=graph["nodes"].index(e)
            if e["id"] == entry["o"]:
                o=graph["nodes"].index(e)
                #update the size of the TO class
                e["value"]=e["value"]+0.1
        
        graph["links"].append( {"source": x, "target": o, "value": 5, "type": entry["type"]} )
        
        

In [72]:
# visualize as force-directed graph in D3
HTML(d3_lib.set_styles(['force_directed_graph']) + 
'<script src="lib/d3/d3.min.js"></script>' + 
      d3_lib.draw_graph('force_directed_graph', {'data': graph}) )

In [73]:
with open('data.json', 'w') as outfile:
    json.dump(graph, outfile)


In [9]:
graph["nodes"]

[{'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0006045',
  'name': 'phosphorus concentration',
  'value': 0.1},
 {'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0006045',
  'name': 'phosphorus content',
  'value': 0.1},
 {'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0002666',
  'name': 'seed phosphorus content',
  'value': 0.1},
 {'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0002666',
  'name': 'phosphorus content',
  'value': 0.1},
 {'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0000645',
  'name': 'relative phosphorus concentration',
  'value': 0.1},
 {'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0000626',
  'name': 'relative phosphorus distribution between shoot and root',
  'value': 0.1},
 {'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0020114',
  'name': 'shoot phosphorus content',
  'value': 0.1},
 {'group': 9,
  'id': 'http://purl.obolibrary.org/obo/TO_0001025',
  'name': 'leaf phosphorus content',
  'value': 0.1}